In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from shutil import copyfile
import ast


dataset_path='../../hotel_with_summ_dataset/'

if not os.path.exists(dataset_path):
	os.mkdir(dataset_path)

processd_path=os.path.join(dataset_path,'processed')
if not os.path.exists(processd_path):
	os.mkdir(processd_path)

In [ ]:
#load original csv data
test=pd.read_csv('../test.csv')
train=pd.read_csv('../train.csv')
dev=pd.read_csv('../dev.csv')


tot=pd.concat([dev,test,train])

In [22]:

#select necessary columns
review_df=tot.sort_values('hotel_url')[['hotel_url','text','rating']]
business_df=tot.sort_values('hotel_url')[['hotel_url']].drop_duplicates(['hotel_url'])

# turn capital to un_capital letter
review_df.loc[:,['hotel_url']]=review_df.loc[:,['hotel_url']].applymap(lambda x:x.lower()).applymap(lambda x:x+'.json')
business_df.loc[:,['hotel_url']]=business_df.loc[:,['hotel_url']].applymap(lambda x:x.lower()).applymap(lambda x:x+'.json')



In [23]:
hotel_with_summs_list=pd.read_csv('../../true_summary/hotel_true_summs.csv')['hotel_url'].to_list()
index=review_df['hotel_url'].isin(hotel_with_summs_list)

In [24]:
#keep only intersection of diego's data 
hotel_with_summs_list=pd.read_csv('../../true_summary/hotel_true_summs.csv')['hotel_url'].to_list()
index=review_df['hotel_url'].isin(hotel_with_summs_list)
review_df=review_df[index]
business_df=business_df[index]

#output to json
review_df.to_json(os.path.join(dataset_path,'review.json'),orient='records')
business_df.to_json(os.path.join(dataset_path,'business.json'),orient='records')
#output to csv
review_df.to_csv(os.path.join(dataset_path,'review.csv'))
business_df.to_csv(os.path.join(dataset_path,'business.csv'))


#original hotel name"Hotel_Review-g28970-d84032-Reviews-The_Graham_Washington_DC_Georgetown_Tapestry_Collection_by_Hilton-Washington_DC_District_review.html.json"
#=>Hotel_Review-g29196-d89254-Reviews-Homewood_Suites_by_Hilton_Atlanta_Alpharetta-Alpharetta_Georgia.html.json

#Diego:"hotel_review-g28970-d84029-reviews-the_georgetown_inn-washington_dc_district_of_columbia.json"
#=>hotel_review-g28970-d84029-reviews-the_georgetown_inn-washington_dc_district_of_columbia.html.json

In [25]:
len(business_df)

1524907

In [26]:
len(tot)

2222373

In [30]:
intersect_hotel_list=business_df['hotel_url'].unique()

In [31]:
len(intersect_hotel_list)

25090

In [33]:
import pickle
with open('../../true_summary/intersect_list.pkl','wb') as file:
    pickle.dump(intersect_hotel_list,file)

### merge true summary and review

In [9]:
true_summary=pd.read_csv('../../true_summary/hotel_true_summs.csv',index_col=0)

In [15]:
review_df=pd.read_csv(os.path.join(dataset_path,'review.csv'),index_col=0)

/opt/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [24]:
pd.merge(review_df.head(),true_summary,on='hotel_url',how='inner')

,hotel_url,text,rating,short_summary,long_summary
0,hotel_review-g10006284-d1083311-reviews-the_re...,The staff is friendly and very service-oriente...,5.0,Excellent luxury hotel. Awesome rooms and serv...,Great pool.Beautiful beach.Great room.Friendly...
1,hotel_review-g10006284-d1083311-reviews-the_re...,We stayed at the Regent Grand for a week in Se...,5.0,Excellent luxury hotel. Awesome rooms and serv...,Great pool.Beautiful beach.Great room.Friendly...
2,hotel_review-g10006284-d1083311-reviews-the_re...,"I took my young son over to Turks and Caicos, ...",5.0,Excellent luxury hotel. Awesome rooms and serv...,Great pool.Beautiful beach.Great room.Friendly...
3,hotel_review-g10006284-d1083311-reviews-the_re...,stayed here for a week in a 3-bedroom condo. A...,5.0,Excellent luxury hotel. Awesome rooms and serv...,Great pool.Beautiful beach.Great room.Friendly...
4,hotel_review-g10006284-d1083311-reviews-the_re...,This is not the best Hotel in turks and caicos...,4.0,Excellent luxury hotel. Awesome rooms and serv...,Great pool.Beautiful beach.Great room.Friendly...


In [25]:
business_df=pd.read_csv(os.path.join(dataset_path,'business.csv'),index_col=0)

/opt/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [31]:
business_df=business_df.drop_duplicates(['hotel_url']).reset_index(drop=True)

In [35]:
business_df=pd.merge(business_df,true_summary,on='hotel_url',how='inner')

In [36]:
business_df.to_csv(os.path.join(dataset_path,'business.csv'))